In [2]:
from gensim.models import Word2Vec
import numpy as np
from utils import *

In [2]:
with open("../data/ft_train.txt", 'r', encoding='utf-8') as f:
    lines = f.readlines()
    ids = [-1 if '-1' in l.split(' ')[0] else 1 for l in lines]
    tweets = [l.strip().split(' ')[1:] for l in lines]

In [7]:
tweets_val = load_tweets('../data/ft_val.txt')

In [8]:
train_embeddings = tweets + [l.strip().split(' ')[1:] for l in tweets_val]

In [20]:
import multiprocessing
cores = multiprocessing.cpu_count()

w2v_model = Word2Vec(min_count=3,
                     window=5,
                     size=100,
                     #sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=5,
                     workers=cores-1,
                     sg=1
                     )

In [21]:
w2v_model.build_vocab(train_embeddings, progress_per=10000)

In [22]:
w2v_model.train(train_embeddings, total_examples=w2v_model.corpus_count, epochs=15)

(360288383, 493927260)

In [23]:
#model.save("../out/word2vec.model")

In [24]:
# add 0 for padding / unknown
word_vectors = np.concatenate((np.zeros((1,w2v_model.wv.vectors.shape[1])),w2v_model.wv.vectors), axis=0)
vocab = {}
i = 1
for w in w2v_model.wv.vocab.keys():
    vocab[w] = i
    i+=1
#assert np.all(word_vectors[vocab['lol']] == w2v_model.wv.vectors[w2v_model.vocabulary['lol']])

In [25]:
DIM = word_vectors.shape[1]
def embed_tweet(tweet):
    embedding = np.zeros((DIM,))
    words = tweet.split()
    c = 0
    for w in words:
        if w in vocab: c += 1
        embedding += word_vectors[vocab.get(w, 0)] # add word vector or only 0s if unknown
    return embedding / c if c > 0 else embedding # return average of (known) words

In [26]:
embedded = np.array([embed_tweet(t) for t in tweets_val])
embedded_train = np.array([embed_tweet(' '.join(t)) for t in tweets])

In [27]:
labels = np.array([1 if '__label__1' in t else -1 for t in tweets_val])

In [28]:
y_train = one_minus_one_labels_to_one_zero(ids)
y_val = one_minus_one_labels_to_one_zero(labels)

In [29]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='saga', multi_class='ovr', verbose=1, random_state=0, n_jobs=-1)
model.fit(embedded_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


convergence after 15 epochs took 49 seconds


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   48.4s finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=0, solver='saga', tol=0.0001,
          verbose=1, warm_start=False)

In [30]:
model.score(embedded, y_val)

0.6576920790051465

In [3]:
tweets_test = load_tweets('')

In [5]:
from keras.models import load_model
model = load_model('../model/mdl_checkpoint_GRU_1_256.hdf5')

/home/anton/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [9]:
def load_tweets(path, contains_ids=False):
    """
    Loads the data from a file. Each line must be either of form `id,tweet` or of form `tweet`.

    Args:
        path (str): Path to the file
        contains_ids (bool): If the file contains ids / labels
    Returns:860
        A list containing the loaded tweets along with a list of the ids/labels if `contains_ids` was True
    """
    lines = []
    with open(path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        lines = [l.strip() for l in lines]
    if contains_ids:
        ids = [int(l.split(',')[0]) for l in lines]
        lines = [l[l.index(',')+1:] for l in lines]
        return lines, ids
    else:
        return lines

X,y = load_tweets('../data/clean_train_full.txt', True)

all_word = []
for elem in X:
    for w in elem.split(' '):
        all_word.append(w)
all_word = list(set(all_word))

vocal_length = len(all_word) + 13

In [10]:
#tokenizer = Tokenizer(num_words=vocal_length)
tokenizer = Tokenizer(num_words=vocal_length, oov_token=1)
tokenizer.fit_on_texts(X)

In [13]:
def load_data(path, test=False):
    lines = []
    with open(path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        lines = [l.strip() for l in lines]
    if test:
        ids = [l.split(',')[0] for l in lines]
        return ids, lines
    else:
        return lines

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


test_data = load_data('../data/test_data.txt')

from tqdm import tqdm

def predict(test_data):
    preds = []
    test_data = tokenizer.texts_to_sequences(test_data)
    test_data = pad_sequences(test_data, 54, padding='post')
    for l in tqdm(test_data):
        preds.append(model.predict(l.reshape(1,-1)))
    return preds

preds = predict(test_data)

predictions = [1 if x > 0.5 else -1 for x in preds]

len(predictions)

 15%|█▌        | 1547/10000 [01:12<06:31, 21.56it/s]

InvalidArgumentError:  indices[0,1] = 389695 is not in [0, 389613)
	 [[node embedding_1/embedding_lookup (defined at /home/anton/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1751) ]] [Op:__inference_keras_scratch_graph_2805]

Function call stack:
keras_scratch_graph


In [ ]:
len(predictions)

 15%|█▌        | 1547/10000 [01:30<06:31, 21.56it/s]

In [ ]:
with open('../out/preds_maggle.csv', 'w') as f:
    f.write('Id,Prediction'+'\n')
    index = 1
    for p in predictions:
        f.write(str(index)+','+str(p)+'\n')
        index += 1